# 2. データ前処理とクリーニング

読み込んだデータを分析用に整形します。

## 2.1 ライブラリのインポート

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# 日本語フォント設定
plt.rcParams['font.sans-serif'] = ['MS Gothic', 'Yu Gothic', 'Hiragino Sans']
plt.rcParams['axes.unicode_minus'] = False

print('ライブラリのインポート完了')

ライブラリのインポート完了


## 2.2 インフルエンザデータの読み込みと整形

現在のデータ構造:
- 行: 保健所（全国、北海道、札幌市、...）
- 列: 週（第01週、第02週、...）

目標の構造:
- 1行が1週のデータ
- 列: 日付、保健所、患者数

In [2]:
def load_influenza_data(years):
    """
    複数年のインフルエンザデータを読み込み、整形する
    """
    all_data = []
    
    for year in years:
        try:
            # データ読み込み
            df = pd.read_csv(f'../data/raw/influenza/{year}.csv', 
                           encoding='shift-jis', skiprows=3, nrows=32)
            
            # 北海道の行だけ抽出（全国は除外）
            hokkaido_row = df[df['保健所名'] == '北海道']
            
            if hokkaido_row.empty:
                print(f'{year}年: 北海道データが見つかりません')
                continue
            
            # 横向きから縦向きに変換（melt）
            hokkaido_row = hokkaido_row.copy()
            
            # 週の列だけ取得（「2019年第01週」のような列名）
            week_cols = [col for col in df.columns if '週' in str(col)]
            
            # データを縦に変換
            melted = hokkaido_row.melt(
                id_vars=['保健所名'],
                value_vars=week_cols,
                var_name='week_label',
                value_name='cases_per_sentinel'
            )
            
            # 週番号を抽出（「2019年第01週」→ 1）
            melted['week'] = melted['week_label'].str.extract(r'第(\d+)週').astype(int)
            melted['year'] = year
            
            # 欠損値を0に置換（'-'や空文字は0とみなす）
            melted['cases_per_sentinel'] = pd.to_numeric(
                melted['cases_per_sentinel'], errors='coerce'
            ).fillna(0)
            
            all_data.append(melted)
            print(f'{year}年: {len(melted)}週分のデータを読み込み')
            
        except Exception as e:
            print(f'{year}年でエラー: {e}')
    
    # 全年データを結合
    df_all = pd.concat(all_data, ignore_index=True)
    
    # 日付を作成（年と週番号から）
    df_all['date'] = df_all.apply(
        lambda row: datetime.strptime(f"{int(row['year'])}-W{int(row['week'])}-1", "%Y-W%W-%w"),
        axis=1
    )
    
    # 必要な列だけ残す
    df_all = df_all[['date', 'year', 'week', 'cases_per_sentinel']].sort_values('date').reset_index(drop=True)
    
    return df_all

In [3]:
# 2015-2024年のデータを読み込み
years = range(2015, 2025)
df_flu = load_influenza_data(years)

print(f'\n整形後のデータ: {df_flu.shape}')
df_flu.head(10)

2015年: 53週分のデータを読み込み
2016年: 52週分のデータを読み込み
2017年: 52週分のデータを読み込み
2018年: 52週分のデータを読み込み
2019年: 52週分のデータを読み込み
2020年: 53週分のデータを読み込み
2021年: 52週分のデータを読み込み
2022年: 52週分のデータを読み込み
2023年: 52週分のデータを読み込み
2024年: 52週分のデータを読み込み

整形後のデータ: (522, 4)


,date,year,week,cases_per_sentinel
0,2015-01-05,2015,1,25.73
1,2015-01-12,2015,2,20.29
2,2015-01-19,2015,3,14.48
3,2015-01-26,2015,4,14.86
4,2015-02-02,2015,5,17.86
5,2015-02-09,2015,6,17.34
6,2015-02-16,2015,7,14.81
7,2015-02-23,2015,8,10.82
8,2015-03-02,2015,9,9.46
9,2015-03-09,2015,10,7.19


In [4]:
# データの基本統計
print('=== 基本統計量 ===')
print(df_flu.describe())

print('\n=== 欠損値 ===')
print(df_flu.isnull().sum())

print('\n=== データ型 ===')
print(df_flu.dtypes)

=== 基本統計量 ===
                                date         year        week  \
count                            522   522.000000  522.000000   
mean   2019-12-28 14:51:02.068965376  2019.492337   26.601533   
min              2015-01-05 00:00:00  2015.000000    1.000000   
25%              2017-06-27 18:00:00  2017.000000   14.000000   
50%              2020-01-02 12:00:00  2019.500000   27.000000   
75%              2022-06-25 06:00:00  2022.000000   40.000000   
max              2024-12-23 00:00:00  2024.000000   53.000000   
std                              NaN     2.876361   15.083206   

       cases_per_sentinel  
count          522.000000  
mean             5.343851  
min              0.000000  
25%              0.010000  
50%              0.255000  
75%              5.720000  
max             60.970000  
std             10.400485  

=== 欠損値 ===
date                  0
year                  0
week                  0
cases_per_sentinel    0
dtype: int64

=== データ型 ===
date        

## 2.3 時系列プロット

In [5]:
def load_weather_data(years):
    """
    複数年の気象データを読み込み、整形する
    """
    all_data = []
    
    for year in years:
        try:
            # 気象庁のCSV構造:
            # 行1: ダウンロード時刻
            # 行2: 空行  
            # 行3: 地点名
            # 行4: 列名
            # 行5: 空行
            # 行6: サブヘッダー
            # 行7～: データ
            
            # header=3で4行目を列名に、skiprows=[0,1,2,4,5]で不要行をスキップ
            df = pd.read_csv(
                f'../data/raw/weather/{year}.csv',
                encoding='shift-jis',
                skiprows=[0, 1, 2, 4, 5]  # 1,2,3,5,6行目をスキップ
            )
            
            all_data.append(df)
            print(f'{year}年: {len(df)}日分のデータを読み込み')
            
        except Exception as e:
            print(f'{year}年でエラー: {e}')
            import traceback
            traceback.print_exc()
    
    if not all_data:
        raise ValueError('データが読み込めませんでした')
    
    # 全年データを結合
    df_all = pd.concat(all_data, ignore_index=True)
    
    # 列名を確認
    print(f'\n=== 読み込んだ列数: {len(df_all.columns)} ===')
    print('最初の10列:')
    for i, col in enumerate(df_all.columns[:10]):
        print(f'  {i}: {col}')
    
    # 日付列（最初の列）
    date_col = df_all.columns[0]
    df_all['date'] = pd.to_datetime(df_all[date_col])
    
    # 気温: 2列目（index=1）、湿度: 11列目（index=10）
    # ※品質情報と均質番号の列があるので、実際のデータは1,4,7,10,...列目
    print(f'\n気温列（2列目）: {df_all.columns[1]}')
    print(f'湿度列（11列目）: {df_all.columns[10]}')
    
    # データを抽出
    df_result = pd.DataFrame({
        'date': df_all['date'],
        'avg_temp': pd.to_numeric(df_all.iloc[:, 1], errors='coerce'),
        'avg_humidity': pd.to_numeric(df_all.iloc[:, 10], errors='coerce')
    })
    
    # 欠損値を確認
    print(f'\n欠損値:')
    print(df_result.isnull().sum())
    
    return df_result

## 2.4 気象データの読み込みと整形

In [6]:
# 気象データを読み込み
df_weather = load_weather_data(years)

print(f'\n気象データ: {df_weather.shape}')
df_weather.head()

2015年: 365日分のデータを読み込み
2016年: 366日分のデータを読み込み
2017年: 365日分のデータを読み込み
2018年: 365日分のデータを読み込み
2019年: 365日分のデータを読み込み
2020年: 366日分のデータを読み込み
2021年: 365日分のデータを読み込み
2022年: 365日分のデータを読み込み
2023年: 365日分のデータを読み込み
2024年: 366日分のデータを読み込み

=== 読み込んだ列数: 20 ===
最初の10列:
  0: 年月日
  1: 平均気温(℃)
  2: 平均気温(℃).1
  3: 平均気温(℃).2
  4: 最高気温(℃)
  5: 最高気温(℃).1
  6: 最高気温(℃).2
  7: 最低気温(℃)
  8: 最低気温(℃).1
  9: 最低気温(℃).2

気温列（2列目）: 平均気温(℃)
湿度列（11列目）: 平均湿度(％)

欠損値:
date            0
avg_temp        0
avg_humidity    0
dtype: int64

気象データ: (3653, 3)


,date,avg_temp,avg_humidity
0,2015-01-01,-4.0,64
1,2015-01-02,-5.5,65
2,2015-01-03,-5.0,52
3,2015-01-04,-2.7,67
4,2015-01-05,-2.4,72


## 2.5 データの保存

整形したデータを保存して、次のNotebookで使えるようにします。

In [7]:
# 整形したデータを保存
df_flu.to_csv('../data/processed/influenza_hokkaido_2015-2024.csv', index=False, encoding='utf-8')
df_weather.to_csv('../data/processed/weather_sapporo_2015-2024.csv', index=False, encoding='utf-8')

print('データを保存しました')
print('- ../data/processed/influenza_hokkaido_2015-2024.csv')
print('- ../data/processed/weather_sapporo_2015-2024.csv')

データを保存しました
- ../data/processed/influenza_hokkaido_2015-2024.csv
- ../data/processed/weather_sapporo_2015-2024.csv


## 次のステップ

次のNotebookでは:
1. 気象データを週次に集計
2. インフルエンザデータと気象データをマージ
3. 特徴量エンジニアリング（ラグ、季節性など）
4. モデル学習の準備